<a href="https://colab.research.google.com/github/liliya2022/ML-with-Python-FreeCodeCamp/blob/main/liliya_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

# Load data
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['Label', 'Message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['Label', 'Message'])

In [ ]:
train_data.head()

In [ ]:
train_text = train_data["Message"].values.tolist()
train_label = np.array([0 if x == 'ham' else 1 for x in train_data["Label"].values.tolist()])
test_text = test_data["Message"].values.tolist()
test_label = np.array([0 if x == 'ham' else 1 for x in test_data["Label"].values.tolist()])

In [ ]:
# Build vocabulary
vocabulary_dict = {}
for text in train_text:
    for word in text.split():
        if word not in vocabulary_dict:
            vocabulary_dict[word] = 1
        else:
            vocabulary_dict[word] += 1

In [ ]:
VOCAB_SIZE = len(vocabulary_dict)
MAX_LENGTH = len(max(train_text, key=lambda p: len(p.split())).split())


In [ ]:
# Encode and pad sequences
padded_train_text = pad_sequences([one_hot(text, VOCAB_SIZE) for text in train_text], maxlen=MAX_LENGTH, padding='post')
padded_test_text = pad_sequences([one_hot(text, VOCAB_SIZE) for text in test_text], maxlen=MAX_LENGTH, padding='post')



In [ ]:
#Build the model
model = Sequential([
    Embedding(VOCAB_SIZE, 100, input_length=MAX_LENGTH),
    Flatten(),
    Dense(64, activation='relu'),  # Increased the number of units
    Dropout(0.5),  # Added dropout to reduce overfitting
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.summary()

In [ ]:
# Set up EarlyStopping
monitor = EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=25, verbose=1, mode='max', restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(padded_train_text, train_label, validation_data=(padded_test_text, test_label),
                    callbacks=[monitor], epochs=1000, verbose=2)


In [ ]:
# Plot Training and Validation Accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    class_dict = {0: 'ham', 1: 'spam'}
    encoded_message = one_hot(pred_text, VOCAB_SIZE)
    padded_message = pad_sequences([encoded_message], maxlen=MAX_LENGTH, padding='post')
    prediction_prob = model.predict(padded_message)[0][0]
    predicted_class = round(prediction_prob)
    return [float(prediction_prob), class_dict[predicted_class]]

# Example usage
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
